In [27]:
import os, io, csv, re
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)

In [ ]:
# Write safety guard: disabled by default
WRITE_ENABLED = False
TMP_DIR = "/Users/clee/Documents/Lab/mlife/data/tmp"
os.makedirs(TMP_DIR, exist_ok=True)

if not hasattr(pd.DataFrame, "_to_csv_original"):
    pd.DataFrame._to_csv_original = pd.DataFrame.to_csv

def _guarded_to_csv(self, path_or_buf=None, *args, **kwargs):
    path_str = str(path_or_buf) if path_or_buf is not None else ""
    if not WRITE_ENABLED:
        print(f"[SKIP WRITE] to_csv disabled. Target: {path_str}")
        return
    if isinstance(path_or_buf, str) and "/data/cleaned/" in path_str:
        print(f"[REDIRECT] cleaned -> tmp for: {path_str}")
        path_str = path_str.replace("/data/cleaned/", "/data/tmp/")
        os.makedirs(os.path.dirname(path_str), exist_ok=True)
        return pd.DataFrame._to_csv_original(self, path_str, *args, **kwargs)
    return pd.DataFrame._to_csv_original(self, path_or_buf, *args, **kwargs)

pd.DataFrame.to_csv = _guarded_to_csv

def enable_writes(enabled: bool = False):
    global WRITE_ENABLED
    WRITE_ENABLED = bool(enabled)
    print("WRITE_ENABLED =", WRITE_ENABLED)

In [28]:
RAW_PATH = "/Users/clee/Documents/Lab/mlife/data/raw/diet_log.csv"
CLEAN_DIR = "/Users/clee/Documents/Lab/mlife/data/cleaned"
os.makedirs(CLEAN_DIR, exist_ok=True)

def _stitch_csv_lines(p, encoding="utf-8"):
    with open(p, "r", encoding=encoding, errors="replace", newline="") as f:
        raw = f.read().splitlines()
    out, buf = [], []
    def balanced(s):
        return s.count('"') % 2 == 0
    for line in raw:
        if not buf:
            buf = [line]
            if balanced(line):
                out.append(line); buf = []
        else:
            buf.append(line)
            if balanced("\n".join(buf)):
                out.append("\n".join(buf)); buf = []
    if buf: out.append("\n".join(buf))
    return out

def read_diet_csv(p: str) -> pd.DataFrame:
    try:
        return pd.read_csv(p)
    except Exception as e1:
        print("Default read_csv failed, trying python engine:", e1)
    try:
        return pd.read_csv(
            p,
            engine="python",
            quotechar='"',
            doublequote=True,
            escapechar='\\',
            on_bad_lines="skip",
        )
    except Exception as e2:
        print("Python engine failed, attempting stitched lines + csv.reader:", e2)
    lines = _stitch_csv_lines(p)
    reader = csv.reader(io.StringIO("\n".join(lines)), delimiter=",", quotechar='"', doublequote=True, escapechar='\\')
    rows = [r for r in reader]
    rows = [r for r in rows if any(cell.strip() for cell in r)]
    if not rows:
        raise ValueError("No rows parsed from CSV.")
    header = rows[0]
    body = rows[1:]
    hlen = len(header)
    fixed = []
    too_short = too_long = 0
    for r in body:
        if len(r) < hlen:
            too_short += 1
            r = r + [""] * (hlen - len(r))
        elif len(r) > hlen:
            too_long += 1
            r = r[:hlen]
        fixed.append(r)
    if too_short or too_long:
        print(f"Adjusted rows -> padded: {too_short}, truncated: {too_long}")
    return pd.DataFrame(fixed, columns=header)

df = read_diet_csv(RAW_PATH)

def canon(s: str) -> str:
    s = str(s).strip().lower()
    s = re.sub(r"[\s\-]+", "_", s)
    s = re.sub(r"[^a-z0-9_]", "", s)
    return s

cols = list(df.columns)
canon_map = {canon(c): c for c in cols}

def find_col(candidates):
    for c in candidates:
        cc = canon(c)
        if cc in canon_map:
            return canon_map[cc]
    return None

DATE_CANDS = ["date", "logged_date", "log_date", "created_at", "added_on", "added on", "timestamp"]
DATE_COL = find_col(DATE_CANDS)
PID_CANDS = ["participant_id", "participant", "user_id", "uid", "subject_id", "pid", "participant id"]
PNAME_CANDS = ["participant_name", "participant name", "name", "user_name", "display_name"]
PID_COL = find_col(PID_CANDS)
PNAME_COL = find_col(PNAME_CANDS)
FOOD_CANDS = ["food_name", "food name", "item_name", "food", "entry_name", "name"]
MEAL_CANDS = ["meal", "meal_type", "meal type", "meal_name", "category"]
QTY_CANDS = ["qty", "quantity", "serving_qty", "serving qty", "amount"]
SERV_CANDS = ["serving_type", "serving type", "serving_unit", "unit", "unit_name"]
FOOD_COL = find_col(FOOD_CANDS)
MEAL_COL = find_col(MEAL_CANDS)
QTY_COL  = find_col(QTY_CANDS)
SERV_COL = find_col(SERV_CANDS)

if DATE_COL is not None:
    df["datetime_raw"] = pd.to_datetime(df[DATE_COL], errors="coerce", utc=False)
    df["date"] = df["datetime_raw"].dt.date
    df["year"] = pd.to_numeric(df["datetime_raw"].dt.year, errors="coerce")
    df["month"] = pd.to_numeric(df["datetime_raw"].dt.month, errors="coerce")
    df["dow"] = df["datetime_raw"].dt.day_name()
else:
    print("No recognizable date column found.")

if PID_COL is None and PNAME_COL is not None:
    df["participant_id"] = df[PNAME_COL].astype(str)
elif PID_COL is not None:
    df["participant_id"] = df[PID_COL].astype(str)
else:
    df["participant_id"] = "unknown"

if FOOD_COL is not None:
    df["food_name"] = df[FOOD_COL].astype(str)
if MEAL_COL is not None:
    df["meal"] = df[MEAL_COL].astype(str).str.strip().str.lower().replace({"b": "breakfast", "breakfast": "breakfast", "l": "lunch", "lnch": "lunch", "d": "dinner", "din": "dinner", "snack": "snack", "snacks": "snack"})
if QTY_COL is not None:
    df["qty"] = pd.to_numeric(df[QTY_COL], errors="coerce")
if SERV_COL is not None:
    df["serving_type"] = df[SERV_COL].astype(str)

subset = [c for c in ["participant_id", "date", "food_name", "qty", "serving_type"] if c in df.columns]
if len(subset) >= 2:
    df = df.drop_duplicates(subset=subset, keep="first")

col_by_canon = {canon(c): c for c in df.columns}

def find_col_by_canon(candidates):
    for c in candidates:
        cc = canon(c)
        if cc in col_by_canon:
            return col_by_canon[cc]
    return None

NUTRIENT_CANDIDATES = {
    "kcal": ["nf calories", "calories", "kcal", "energy_kcal", "og kcal"],
    "protein_g": ["nf protein", "protein_g", "protein"],
    "carbs_g": ["nf total carbohydrate", "carbs_g", "carbohydrates_g", "carbs"],
    "fat_g": ["nf total fat", "fat_g", "total_fat_g", "fat"],
    "sugars_g": ["nf sugars", "sugars_g", "sugar_g", "sugar"],
    "fiber_g": ["nf dietary fiber", "fiber_g", "dietary_fiber_g", "fiber"],
    "sodium_mg": ["nf sodium", "sodium_mg", "sodium"],
    "cholesterol_mg": ["nf cholesterol", "cholesterol_mg", "cholesterol"],
    "potassium_mg": ["nf potassium", "potassium_mg", "potassium"],
}

for std_col, cands in NUTRIENT_CANDIDATES.items():
    src = find_col_by_canon(cands)
    if src is None:
        df[std_col] = np.nan
    else:
        df[std_col] = pd.to_numeric(df[src], errors="coerce")

keep_cols = []
for c in ["participant_id", "datetime_raw", "date", "year", "month", "dow", "meal", "food_name", "qty", "serving_type"]:
    if c in df.columns:
        keep_cols.append(c)
keep_cols += [c for c in ["kcal", "protein_g", "carbs_g", "fat_g", "sugars_g", "fiber_g", "sodium_mg", "cholesterol_mg", "potassium_mg"] if c in df.columns]

rowlevel_out = os.path.join(CLEAN_DIR, "diet_log_rowlevel_cleaned_v2.csv")
df[keep_cols].to_csv(rowlevel_out, index=False)

base = df.copy()
if "date" not in base.columns and "datetime_raw" in base.columns:
    base["date"] = base["datetime_raw"].dt.date

agg_group = [c for c in ["participant_id", "date"] if c in base.columns]
if len(agg_group) < 2:
    raise ValueError("Need participant_id and date to create daily aggregates.")

num_cols = [c for c in ["kcal", "protein_g", "carbs_g", "fat_g", "sugars_g", "fiber_g", "sodium_mg", "cholesterol_mg", "potassium_mg", "qty"] if c in base.columns]

daily = base.groupby(agg_group)[num_cols].sum(min_count=1).reset_index()

if "kcal" in daily.columns:
    kcal = daily["kcal"].replace(0, np.nan)
    if "carbs_g" in daily and "fat_g" in daily and "protein_g" in daily:
        kcal_from_macros = daily["carbs_g"]*4 + daily["protein_g"]*4 + daily["fat_g"]*9
        daily["macro_kcal_ratio"] = (kcal_from_macros / kcal).clip(upper=5)
        daily["carb_pct_kcal"] = (daily["carbs_g"]*4 / kcal).clip(upper=1)
        daily["protein_pct_kcal"] = (daily["protein_g"]*4 / kcal).clip(upper=1)
        daily["fat_pct_kcal"] = (daily["fat_g"]*9 / kcal).clip(upper=1)

if "food_name" in base.columns:
    item_counts = base.groupby(agg_group)["food_name"].count().rename("item_count").reset_index()
    daily = daily.merge(item_counts, on=agg_group, how="left")

if "meal" in base.columns:
    meal_ct = base.assign(_cnt=1).pivot_table(index=agg_group, columns="meal", values="_cnt", aggfunc="sum", fill_value=0).add_prefix("meal_").reset_index()
    daily = daily.merge(meal_ct, on=agg_group, how="left")

ml_out = os.path.join(CLEAN_DIR, "diet_log_daily_ml_ready.csv")
daily.to_csv(ml_out, index=False)

summary_cols = [c for c in ["kcal", "protein_g", "carbs_g", "fat_g", "sugars_g", "fiber_g", "item_count"] if c in daily.columns]
print("Participants:", daily["participant_id"].nunique())
print("Date range:", daily.get("date").min(), "->", daily.get("date").max())
daily[summary_cols].describe().T

Default read_csv failed, trying python engine: Error tokenizing data. C error: Expected 35 fields in line 8, saw 50

Participants: 106
Date range: 1969-12-31 -> 2023-07-06
Participants: 106
Date range: 1969-12-31 -> 2023-07-06


,count,mean,std,min,25%,50%,75%,max
kcal,18824.0,1217.221722,3030.332488,0.0,802.675,1172.65,1508.65,404259.3
protein_g,18824.0,57.134658,338.486254,0.0,34.500,52.20,70.90,46245.2
carbs_g,18824.0,132.191500,202.513794,0.0,75.600,120.80,171.00,24630.3
fat_g,18824.0,53.337665,106.050420,0.0,30.500,48.80,68.30,13497.2
sugars_g,18824.0,47.566309,48.789604,0.0,19.300,37.80,64.00,2358.7
fiber_g,18824.0,13.963908,34.812875,0.0,6.500,11.20,17.30,4112.4
item_count,18824.0,7.534530,4.177410,1.0,4.000,7.00,10.00,33.0


In [ ]:
def _stitch_csv_lines(p, encoding="utf-8"):
    with open(p, "r", encoding=encoding, errors="replace", newline="") as f:
        raw = f.read().splitlines()
    out, buf = [], []
    def balanced(s):
        return s.count('"') % 2 == 0
    for line in raw:
        if not buf:
            buf = [line]
            if balanced(line):
                out.append(line); buf = []
        else:
            buf.append(line)
            if balanced("\n".join(buf)):
                out.append("\n".join(buf)); buf = []
    if buf: out.append("\n".join(buf))
    return out


def read_diet_csv(p: str) -> pd.DataFrame:
    try:
        return pd.read_csv(p)
    except Exception as e1:
        print("Default read_csv failed, trying python engine:", e1)
    try:
        return pd.read_csv(
            p,
            engine="python",
            quotechar='"',
            doublequote=True,
            escapechar='\\',
            on_bad_lines="skip",
        )
    except Exception as e2:
        print("Python engine failed, attempting stitched lines + csv.reader:", e2)
    lines = _stitch_csv_lines(p)
    reader = csv.reader(io.StringIO("\n".join(lines)), delimiter=",", quotechar='"', doublequote=True, escapechar='\\')
    rows = [r for r in reader]
    rows = [r for r in rows if any(cell.strip() for cell in r)]
    if not rows:
        raise ValueError("No rows parsed from CSV.")
    header = rows[0]
    body = rows[1:]
    hlen = len(header)
    fixed = []
    too_short = too_long = 0
    for r in body:
        if len(r) < hlen:
            too_short += 1
            r = r + [""] * (hlen - len(r))
        elif len(r) > hlen:
            too_long += 1
            r = r[:hlen]
        fixed.append(r)
    if too_short or too_long:
        print(f"Adjusted rows -> padded: {too_short}, truncated: {too_long}")
    return pd.DataFrame(fixed, columns=header)


df = read_diet_csv(RAW_PATH)
print("Loaded shape:", df.shape)
df.head(3)

Default read_csv failed, trying python engine: Error tokenizing data. C error: Expected 35 fields in line 8, saw 50

Loaded shape: (148242, 35)
Loaded shape: (148242, 35)


,Diet Log ID,Participant name,Participant ID,Group,Date,Meal,Food Name,Thumb,Brand Name,Qty,Serving Type,Nf Calories,Nf Total Fat,Nf Saturated Fat,Nf Cholesterol,Nf Sodium,Nf Total Carbohydrate,Nf Dietary Fiber,Nf Sugars,Nf Protein,Nf Potassium,Nf Added Sugars,Nf Trans Fatty Acid,Added On,Og Serving Weight Grams,Og Kcal,Og Unit,Sel Serving Qty,Sel Serving Weight Grams,Alt Measures,Full Nutrients,Nix Brand Id,Nix Brand Name,Nix Item Id,Nix Item Name
0,152340,user user 10,10,mLife+Points,06/30/2023,Lunch,deli turkey,https://nix-tag-images.s3.amazonaws.com/855_th...,NaN,3,oz,95.3,2.6,0.1,46.8,1020.6,6.5,0.3,3.4,11.5,296.8,11.5,11.5,06/30/2023 10:20:46 pm,56.7,63.50,oz,2.00,56.7,"[{""serving_weight"":48,""measure"":""oz (1 serving...","[{""attr_id"":203,""value"":7.6545},{""attr_id"":204...",NaN,NaN,NaN,NaN
1,152341,user user 10,10,mLife+Points,06/30/2023,Lunch,fruit salad,https://nix-tag-images.s3.amazonaws.com/1773_t...,NaN,2,cup,193.2,0.9,0.1,0,5.3,49,6.5,32.9,2.7,728.3,2.7,0,06/30/2023 10:20:46 pm,162.4,96.62,cup,1.00,162.4,"[{""serving_weight"":162.4,""measure"":""cup"",""seq""...","[{""attr_id"":203,""value"":1.3555},{""attr_id"":204...",NaN,NaN,NaN,NaN
2,152342,user user 10,10,mLife+Points,06/30/2023,Dinner,hummus,https://nix-tag-images.s3.amazonaws.com/636_th...,NaN,0.25,cup,99.6,5.8,0.9,0,227.4,8.6,3.6,4.7,4.7,136.8,4.7,4.7,06/30/2023 10:26:14 pm,60.0,99.60,cup,0.25,60.0,"[{""serving_weight"":15,""measure"":""tbsp"",""seq"":1...","[{""attr_id"":203,""value"":4.74},{""attr_id"":204,""...",NaN,NaN,NaN,NaN


In [ ]:
print("Columns (", len(df.columns), "):\n", list(df.columns))
print("\nDTypes:\n", df.dtypes)
non_null = df.notnull().sum().sort_values(ascending=False)
print("\nTop non-null columns:\n", non_null.head(20))
mem_mb = df.memory_usage(deep=True).sum() / (1024**2)
print(f"\nApprox memory usage: {mem_mb:.2f} MB")

Columns ( 35 ):
 ['Diet Log ID', 'Participant name', 'Participant ID', 'Group', 'Date', 'Meal', 'Food Name', 'Thumb', 'Brand Name', 'Qty', 'Serving Type', 'Nf Calories', 'Nf Total Fat', 'Nf Saturated Fat', 'Nf Cholesterol', 'Nf Sodium', 'Nf Total Carbohydrate', 'Nf Dietary Fiber', 'Nf Sugars', 'Nf Protein', 'Nf Potassium', 'Nf Added Sugars', 'Nf Trans Fatty Acid', 'Added On', 'Og Serving Weight Grams', 'Og Kcal', 'Og Unit', 'Sel Serving Qty', 'Sel Serving Weight Grams', 'Alt Measures', 'Full Nutrients', 'Nix Brand Id', 'Nix Brand Name', 'Nix Item Id', 'Nix Item Name']

DTypes:
 Diet Log ID                   int64
Participant name             object
Participant ID                int64
Group                        object
Date                         object
Meal                         object
Food Name                    object
Thumb                        object
Brand Name                   object
Qty                          object
Serving Type                 object
Nf Calories        

In [ ]:
miss_ct = df.isna().sum()
miss_pct = (df.isna().mean() * 100)
missing = pd.DataFrame({"missing": miss_ct, "missing_%": miss_pct}).sort_values("missing_%", ascending=False)
missing[missing["missing"] > 0].head(40)

,missing,missing_%
Brand Name,110837,74.767610
Nix Item Name,107860,72.759407
Nix Item Id,107859,72.758732
Nix Brand Name,107859,72.758732
Nix Brand Id,107859,72.758732
Alt Measures,37954,25.602731
Og Unit,21742,14.666559
Og Serving Weight Grams,1206,0.813535
Sel Serving Weight Grams,1206,0.813535
Qty,31,0.020912


In [ ]:
import re

def canon(s: str) -> str:
    s = str(s).strip().lower()
    s = re.sub(r"[\s\-]+", "_", s)
    s = re.sub(r"[^a-z0-9_]", "", s)
    return s
cols = list(df.columns)
canon_map = {canon(c): c for c in cols}

def find_col(candidates):
    for c in candidates:
        cc = canon(c)
        if cc in canon_map:
            return canon_map[cc]
    return None
DATE_CANDS = ["date", "logged_date", "log_date", "created_at", "added_on", "added on", "timestamp"]
DATE_COL = find_col(DATE_CANDS)
PID_CANDS = ["participant_id", "participant", "user_id", "uid", "subject_id", "pid", "participant id"]
PNAME_CANDS = ["participant_name", "participant name", "name", "user_name", "display_name"]
PID_COL = find_col(PID_CANDS)
PNAME_COL = find_col(PNAME_CANDS)
FOOD_CANDS = ["food_name", "food name", "item_name", "food", "entry_name", "name"]
MEAL_CANDS = ["meal", "meal_type", "meal type", "meal_name", "category"]
QTY_CANDS = ["qty", "quantity", "serving_qty", "serving qty", "amount"]
SERV_CANDS = ["serving_type", "serving type", "serving_unit", "unit", "unit_name"]
FOOD_COL = find_col(FOOD_CANDS)
MEAL_COL = find_col(MEAL_CANDS)
QTY_COL  = find_col(QTY_CANDS)
SERV_COL = find_col(SERV_CANDS)
print({"DATE_COL": DATE_COL, "PID_COL": PID_COL, "PNAME_COL": PNAME_COL, "FOOD_COL": FOOD_COL, "MEAL_COL": MEAL_COL, "QTY_COL": QTY_COL, "SERV_COL": SERV_COL})

{'DATE_COL': 'Date', 'PID_COL': 'participant_id', 'PNAME_COL': 'Participant name', 'FOOD_COL': 'Food Name', 'MEAL_COL': 'Meal', 'QTY_COL': 'Qty', 'SERV_COL': 'Serving Type'}


In [ ]:
if DATE_COL is not None:
    df["datetime_raw"] = pd.to_datetime(df[DATE_COL], errors="coerce", utc=False)
    df["date"] = df["datetime_raw"].dt.date
    df["year"] = pd.to_numeric(df["datetime_raw"].dt.year, errors="coerce")
    df["month"] = pd.to_numeric(df["datetime_raw"].dt.month, errors="coerce")
    df["dow"] = df["datetime_raw"].dt.day_name()
    print("Date range:", df["datetime_raw"].min(), "->", df["datetime_raw"].max())
    print("Unparsed datetime rows:", df["datetime_raw"].isna().sum())
else:
    print("No recognizable date column found.")

Date range: 1969-12-31 00:00:00 -> 2023-07-06 00:00:00
Unparsed datetime rows: 14


In [ ]:
if PID_COL is None and PNAME_COL is not None:
    df["participant_id"] = df[PNAME_COL].astype(str)
elif PID_COL is not None:
    df["participant_id"] = df[PID_COL].astype(str)
else:
    df["participant_id"] = "unknown"
n_ids = df["participant_id"].nunique(dropna=True)
print("Unique participants:", n_ids)

Unique participants: 1


In [ ]:
if FOOD_COL is not None: df["food_name"] = df[FOOD_COL].astype(str)
if MEAL_COL is not None:
    df["meal"] = df[MEAL_COL].astype(str).str.strip().str.lower().replace({"b": "breakfast", "breakfast": "breakfast", "l": "lunch", "lnch": "lunch", "d": "dinner", "din": "dinner", "snack": "snack", "snacks": "snack"})
if QTY_COL is not None:
    df["qty"] = pd.to_numeric(df[QTY_COL], errors="coerce")
if SERV_COL is not None:
    df["serving_type"] = df[SERV_COL].astype(str)
for col in ["food_name", "meal", "qty", "serving_type"]:
    if col in df.columns:
        print(col, "-> non-null:", df[col].notna().sum())

food_name -> non-null: 148242
meal -> non-null: 148242
qty -> non-null: 147970
serving_type -> non-null: 148242
serving_type -> non-null: 148242


In [ ]:
meal_map = {"breakfast": 1, "lunch": 2, "dinner": 3, "snack": 4}
if "meal" in df.columns:
    df["meal_code"] = (
        df["meal"].astype(str).str.strip().str.lower().map(meal_map).astype("Int64")
    )
    print(df["meal_code"].value_counts(dropna=False).sort_index())

In [ ]:
subset = [c for c in ["participant_id", "date", "food_name", "qty", "serving_type"] if c in df.columns]
if len(subset) >= 2:
    before = len(df)
    df = df.drop_duplicates(subset=subset, keep="first")
    print(f"Deduplicated {before - len(df)} rows using subset={subset}")
else:
    print("Not enough normalized columns to deduplicate; skipping.")

Deduplicated 17039 rows using subset=['participant_id', 'date', 'food_name', 'qty', 'serving_type']


In [ ]:
try:
    canon
except NameError:
    import re
    def canon(s: str) -> str:
        s = str(s).strip().lower()
        s = re.sub(r"[\s\-]+", "_", s)
        s = re.sub(r"[^a-z0-9_]", "", s)
        return s

col_by_canon = {canon(c): c for c in df.columns}

def find_col_by_canon(candidates):
    for c in candidates:
        cc = canon(c)
        if cc in col_by_canon:
            return col_by_canon[cc]
    return None

NUTRIENT_CANDIDATES = {
    "kcal": ["nf calories", "calories", "kcal", "energy_kcal", "og kcal"],
    "protein_g": ["nf protein", "protein_g", "protein"],
    "carbs_g": ["nf total carbohydrate", "carbs_g", "carbohydrates_g", "carbs"],
    "fat_g": ["nf total fat", "fat_g", "total_fat_g", "fat"],
    "sugars_g": ["nf sugars", "sugars_g", "sugar_g", "sugar"],
    "fiber_g": ["nf dietary fiber", "fiber_g", "dietary_fiber_g", "fiber"],
    "sodium_mg": ["nf sodium", "sodium_mg", "sodium"],
    "cholesterol_mg": ["nf cholesterol", "cholesterol_mg", "cholesterol"],
    "potassium_mg": ["nf potassium", "potassium_mg", "potassium"],
}

for std_col, cands in NUTRIENT_CANDIDATES.items():
    src = find_col_by_canon(cands)
    if src is None:
        df[std_col] = np.nan
    else:
        df[std_col] = pd.to_numeric(df[src], errors="coerce")

nutrient_cols = list(NUTRIENT_CANDIDATES.keys())
df[nutrient_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
kcal,131195.0,168.095174,1138.713233,0.0,50.0,114.1,215.6,403650.0
protein_g,131195.0,7.935584,128.226399,0.0,0.9,3.4,9.4,46215.0
carbs_g,131195.0,18.105401,74.157866,0.0,2.0,11.0,26.1,24570.0
fat_g,131195.0,7.408812,39.613671,0.0,0.3,3.5,10.3,13455.0
sugars_g,131195.0,6.478661,15.904234,0.0,0.2,2.0,7.5,2340.0
fiber_g,131195.0,1.901895,12.885385,0.0,0.0,0.9,2.4,4095.0
sodium_mg,131195.0,274.020421,2484.726208,0.0,15.0,106.7,320.0,877500.0
cholesterol_mg,131195.0,28.239123,69.575463,0.0,0.0,0.0,25.0,3906.0
potassium_mg,131195.0,191.457604,331.926578,0.0,21.5,102.6,257.1,47775.0


In [ ]:
keep_cols = []
for c in ["participant_id", "datetime_raw", "date", "year", "month", "dow", "meal", "food_name", "qty", "serving_type"]:
    if c in df.columns: keep_cols.append(c)
keep_cols += [c for c in ["kcal", "protein_g", "carbs_g", "fat_g", "sugars_g", "fiber_g", "sodium_mg", "cholesterol_mg", "potassium_mg"] if c in df.columns]
rowlevel_out = os.path.join(CLEAN_DIR, "diet_log_rowlevel_cleaned_v2.csv")
df[keep_cols].to_csv(rowlevel_out, index=False)
print("Wrote:", rowlevel_out, "with", len(df), "rows and", len(keep_cols), "cols")

Wrote: /Users/clee/Documents/Lab/mlife/data/cleaned/diet_log_rowlevel_cleaned_v2.csv with 131203 rows and 19 cols


In [ ]:
base = df.copy()
if "date" not in base.columns and "datetime_raw" in base.columns:
    base["date"] = base["datetime_raw"].dt.date
agg_group = [c for c in ["participant_id", "date"] if c in base.columns]
if len(agg_group) < 2:
    raise ValueError("Need participant_id and date to create daily aggregates.")
num_cols = [c for c in ["kcal", "protein_g", "carbs_g", "fat_g", "sugars_g", "fiber_g", "sodium_mg", "cholesterol_mg", "potassium_mg", "qty"] if c in base.columns]
daily = base.groupby(agg_group)[num_cols].sum(min_count=1).reset_index()
if "kcal" in daily.columns:
    kcal = daily["kcal"].replace(0, np.nan)
    if "carbs_g" in daily and "fat_g" in daily and "protein_g" in daily:
        kcal_from_macros = daily["carbs_g"]*4 + daily["protein_g"]*4 + daily["fat_g"]*9
        daily["macro_kcal_ratio"] = (kcal_from_macros / kcal).clip(upper=5)
        daily["carb_pct_kcal"] = (daily["carbs_g"]*4 / kcal).clip(upper=1)
        daily["protein_pct_kcal"] = (daily["protein_g"]*4 / kcal).clip(upper=1)
        daily["fat_pct_kcal"] = (daily["fat_g"]*9 / kcal).clip(upper=1)
if "food_name" in base.columns:
    item_counts = base.groupby(agg_group)["food_name"].count().rename("item_count").reset_index()
    daily = daily.merge(item_counts, on=agg_group, how="left")
if "meal" in base.columns:
    meal_ct = base.assign(_cnt=1).pivot_table(index=agg_group, columns="meal", values="_cnt", aggfunc="sum", fill_value=0).add_prefix("meal_").reset_index()
    daily = daily.merge(meal_ct, on=agg_group, how="left")
ml_out = os.path.join(CLEAN_DIR, "diet_log_daily_ml_ready.csv")
daily.to_csv(ml_out, index=False)
print("Wrote:", ml_out, "with", len(daily), "rows and", daily.shape[1], "cols")
daily.head(10)

keep_cols = []
for c in ["participant_id", "datetime_raw", "date", "year", "month", "dow", "meal", "meal_code", "food_name", "qty", "serving_type"]:
    if c in df.columns: keep_cols.append(c)
keep_cols += [c for c in ["kcal", "protein_g", "carbs_g", "fat_g", "sugars_g", "fiber_g", "sodium_mg", "cholesterol_mg", "potassium_mg"] if c in df.columns]
rowlevel_out = os.path.join(CLEAN_DIR, "diet_log_rowlevel_cleaned_v2.csv")
df[keep_cols].to_csv(rowlevel_out, index=False)
print("Wrote:", rowlevel_out, "with", len(df), "rows and", len(keep_cols), "cols")

Wrote: /Users/clee/Documents/Lab/mlife/data/cleaned/diet_log_daily_ml_ready.csv with 377 rows and 22 cols


,participant_id,date,kcal,protein_g,carbs_g,fat_g,sugars_g,fiber_g,sodium_mg,cholesterol_mg,potassium_mg,qty,macro_kcal_ratio,carb_pct_kcal,protein_pct_kcal,fat_pct_kcal,item_count,meal_breakfast,meal_dinner,meal_lunch,meal_nan,meal_snack
0,unknown,1969-12-31,7761.2,401.6,707.3,376.7,238.6,53.2,14246.7,1557.9,7576.6,625.590,1.008336,0.364531,0.206978,0.436827,38,6,20,8,0,4
1,unknown,2022-06-25,1492.3,68.1,135.3,82.7,63.7,30.6,2770.6,214.6,1174.8,10.830,1.043959,0.362662,0.182537,0.498760,8,1,4,3,0,0
2,unknown,2022-06-26,5597.8,304.0,575.2,215.5,296.9,68.1,7824.4,1168.6,8455.9,41.570,0.974722,0.411019,0.217228,0.346475,26,5,10,9,0,2
3,unknown,2022-06-27,76323.4,3720.3,7654.7,3450.3,2943.2,745.9,112184.6,11969.1,86952.7,1177.100,1.003004,0.401172,0.194976,0.406857,378,99,120,95,1,63
4,unknown,2022-06-28,103483.5,5063.5,10544.3,4557.7,3788.7,1189.6,171967.1,17989.5,142221.0,3237.981,0.999681,0.407574,0.195722,0.396385,692,182,191,194,0,125
5,unknown,2022-06-29,125146.7,5814.3,12913.9,5642.8,4557.4,1406.7,212468.5,20839.4,162620.1,3917.350,1.004405,0.412760,0.185839,0.405805,772,216,232,181,0,143
6,unknown,2022-06-30,124635.8,5487.5,14049.7,5342.1,5726.4,1397.0,212183.6,18688.9,151778.3,4081.410,1.012772,0.450904,0.176113,0.385755,763,175,232,221,0,135
7,unknown,2022-07-01,128070.9,5198.1,14714.9,5611.9,5953.7,1380.4,199886.2,17132.4,132123.1,4928.740,1.016305,0.459586,0.162351,0.394368,745,182,210,210,0,143
8,unknown,2022-07-02,127526.7,5294.3,13406.0,5606.9,5003.4,1260.3,215925.4,20092.9,137815.7,4185.996,0.982252,0.420492,0.166061,0.395698,724,202,232,166,0,124
9,unknown,2022-07-03,113521.4,5403.3,11748.4,4870.8,4503.2,1097.3,186426.8,20992.1,140251.0,2558.220,0.990509,0.413962,0.190389,0.386158,706,182,209,198,0,117


In [ ]:
summary_cols = [c for c in ["kcal", "protein_g", "carbs_g", "fat_g", "sugars_g", "fiber_g", "item_count"] if c in daily.columns]
print("Participants:", daily["participant_id"].nunique())
print("Date range:", daily.get("date").min(), "->", daily.get("date").max())
daily[summary_cols].describe().T

Participants: 1
Date range: 1969-12-31 -> 2023-07-06


,count,mean,std,min,25%,50%,75%,max
kcal,377.0,58490.201857,33177.982282,1200.6,41978.8,50746.1,68713.8,526307.1
protein_g,377.0,2761.363926,2762.950245,54.8,1891.2,2315.9,3209.0,51880.4
carbs_g,377.0,6299.752255,2962.748946,135.3,4457.2,5480.1,7351.0,36914.5
fat_g,377.0,2577.991512,1323.449916,36.0,1891.1,2243.0,3040.8,19114.9
sugars_g,377.0,2254.198939,999.256655,32.6,1611.9,1966.1,2647.7,7446.7
fiber_g,377.0,661.779045,397.951104,30.6,424.7,531.5,804.2,5383.8
item_count,377.0,347.981432,151.354530,8.0,246.0,295.0,433.0,772.0


In [29]:
yrs_df = df.copy()
if "year" not in yrs_df.columns:
    if "date" in yrs_df.columns:
        yrs_df["year"] = pd.to_datetime(yrs_df["date"], errors="coerce").dt.year
    elif "datetime_raw" in yrs_df.columns:
        yrs_df["year"] = pd.to_datetime(yrs_df["datetime_raw"], errors="coerce").dt.year
row_counts = (
    yrs_df.dropna(subset=["year"]).groupby("year").size().rename("row_count").reset_index().sort_values("year")
)
print("Row-level rows per year:")
print(row_counts.to_string(index=False))

daily_year = None
if "date" in daily.columns:
    daily_year = pd.to_datetime(daily["date"], errors="coerce").dt.year
elif "datetime_raw" in daily.columns:
    daily_year = pd.to_datetime(daily["datetime_raw"], errors="coerce").dt.year
if daily_year is not None:
    daily_counts = (
        daily.assign(year=daily_year).dropna(subset=["year"]).groupby("year").size().rename("daily_row_count").reset_index().sort_values("year")
    )
    print("Daily rows per year:")
    print(daily_counts.to_string(index=False))
else:
    print("No daily date available for yearly counts.")

Row-level rows per year:
  year  row_count
1969.0         39
2022.0      93827
2023.0      47964
Daily rows per year:
 year  daily_row_count
 1969                8
 2022            11943
 2023             6873


In [30]:
weird_dir = "/Users/clee/Documents/Lab/mlife/data/weird/diet_log"
os.makedirs(weird_dir, exist_ok=True)
df1969 = df.copy()
if "year" not in df1969.columns:
    if "date" in df1969.columns:
        df1969["year"] = pd.to_datetime(df1969["date"], errors="coerce").dt.year
    elif "datetime_raw" in df1969.columns:
        df1969["year"] = pd.to_datetime(df1969["datetime_raw"], errors="coerce").dt.year
df1969 = df1969[df1969["year"] == 1969]
out_path = os.path.join(weird_dir, "diet_log_year_1969_rows.csv")
df1969.to_csv(out_path, index=False)
print("Wrote:", out_path, "rows:", len(df1969))

Wrote: /Users/clee/Documents/Lab/mlife/data/weird/diet_log/diet_log_year_1969_rows.csv rows: 39


In [31]:
weird_dir = "/Users/clee/Documents/Lab/mlife/data/weird/diet_log"
os.makedirs(weird_dir, exist_ok=True)
if "datetime_raw" in df.columns:
    dt_series = df["datetime_raw"]
elif "date" in df.columns:
    dt_series = pd.to_datetime(df["date"], errors="coerce")
else:
    dt_series = pd.Series([pd.NaT] * len(df))
df_unparsed = df[dt_series.isna()].copy()
out_path = os.path.join(weird_dir, "unparsed_datetime_rows.csv")
df_unparsed.to_csv(out_path, index=False)
print("Wrote:", out_path, "rows:", len(df_unparsed))

Wrote: /Users/clee/Documents/Lab/mlife/data/weird/diet_log/unparsed_datetime_rows.csv rows: 14


In [32]:
try:
    canon
except NameError:
    def canon(s: str) -> str:
        s = str(s).strip().lower()
        s = re.sub(r"[\s\-]+", "_", s)
        s = re.sub(r"[^a-z0-9_]", "", s)
        return s
colmap = {canon(c): c for c in df.columns}

def find_col(cands):
    for c in cands:
        cc = canon(c)
        if cc in colmap:
            return colmap[cc]
    return None
group_col = find_col(["group", "participant_group", "arm", "cohort"]) or "group"
summary = {}
summary["rows"] = len(df)
summary["daily_rows"] = len(daily) if "daily" in globals() else None
summary["participants"] = df["participant_id"].nunique() if "participant_id" in df.columns else None
print("Summary:", summary)
if isinstance(group_col, str) and group_col in df.columns:
    grp_rows = df.groupby(group_col).size().rename("row_count").reset_index().sort_values("row_count", ascending=False)
    grp_parts = df.groupby(group_col)["participant_id"].nunique().rename("unique_participants").reset_index()
    grp = grp_rows.merge(grp_parts, on=group_col, how="left")
    print("Rows and unique participants by group:")
    print(grp.to_string(index=False))
else:
    print("No group column found.")
if "meal" in df.columns:
    meal_counts = df["meal"].value_counts(dropna=False).rename_axis("meal").reset_index(name="row_count")
    print("Rows by meal:")
    print(meal_counts.to_string(index=False))
if "participant_id" in df.columns:
    per_part_rows = df.groupby("participant_id").size().describe()
    print("Per-participant row counts describe:")
    print(per_part_rows.to_string())
if "participant_id" in daily.columns:
    per_part_days = daily.groupby("participant_id").size().describe()
    print("Per-participant daily counts describe:")
    print(per_part_days.to_string())
key_nutrients = [c for c in ["kcal","protein_g","carbs_g","fat_g","sugars_g","fiber_g"] if c in df.columns]
if key_nutrients:
    avail = df[key_nutrients].notna().mean().mul(100).round(2)
    print("Row-level nutrient availability (% non-missing):")
    print(avail.to_string())
key_nutrients_daily = [c for c in ["kcal","protein_g","carbs_g","fat_g","sugars_g","fiber_g"] if c in daily.columns]
if key_nutrients_daily:
    avail_d = daily[key_nutrients_daily].notna().mean().mul(100).round(2)
    print("Daily nutrient availability (% non-missing):")
    print(avail_d.to_string())
if isinstance(group_col, str) and group_col in df.columns and "meal" in df.columns:
    gm = pd.crosstab(df[group_col], df["meal"]) 
    print("Group x Meal matrix:")
    print(gm.to_string())

Summary: {'rows': 141844, 'daily_rows': 18824, 'participants': 106}
Rows and unique participants by group:
          Group  row_count  unique_participants
   mLife+Points      87051                   54
mLife (control)      54793                   52
Rows by meal:
     meal  row_count
breakfast      42265
   dinner      41437
    lunch      37777
    snack      20357
      nan          8
Per-participant row counts describe:
count     106.000000
mean     1338.150943
std      1270.610995
min         4.000000
25%       366.250000
50%       791.500000
75%      2172.000000
max      4823.000000
Per-participant daily counts describe:
count    106.000000
mean     177.584906
std      136.094006
min        1.000000
25%       50.500000
50%      138.500000
75%      345.000000
max      374.000000
Row-level nutrient availability (% non-missing):
kcal         99.99
protein_g    99.99
carbs_g      99.99
fat_g        99.99
sugars_g     99.99
fiber_g      99.99
Daily nutrient availability (% non-missing

In [33]:
cols_info = []
for c in df.columns:
    s = df[c]
    cols_info.append({
        "column": c,
        "dtype": str(s.dtype),
        "non_null": int(s.notna().sum()),
        "missing_%": round(float(s.isna().mean()*100), 2),
        "unique": int(s.nunique(dropna=True)),
    })
info_df = pd.DataFrame(cols_info).sort_values(["missing_%", "column"], ascending=[False, True])
print("All columns (", len(df.columns), "):")
print(info_df.to_string(index=False))
canon_map_cols = {c: c.strip().lower() for c in df.columns}
pattern_drop = [c for c in df.columns if canon_map_cols[c].startswith("nix")] 
explicit_drop = [c for c in df.columns if canon_map_cols[c] in {"brand name", "thumb", "alt measures", "full nutrients"}]
high_missing_drop = info_df.loc[info_df["missing_%"] >= 95, "column"].tolist()
proposed_drop = sorted(set(pattern_drop + explicit_drop + high_missing_drop))
print("Proposed drop columns:")
print(proposed_drop)
print("Note: cleaned outputs already omit non-essential columns via keep_cols.")

All columns ( 54 ):
                  column          dtype  non_null  missing_%  unique
              Brand Name         object     36198      74.48    3384
            Nix Brand Id         object     39132      72.41    3411
          Nix Brand Name         object     39132      72.41    3412
             Nix Item Id         object     39132      72.41   10699
           Nix Item Name         object     39131      72.41   10262
            Alt Measures         object    105104      25.90    5550
                 Og Unit         object    121004      14.69    1281
 Og Serving Weight Grams        float64    140680       0.82    2421
Sel Serving Weight Grams        float64    140680       0.82    2702
                     qty        float64    141573       0.19     720
                     Qty         object    141813       0.02     936
                    Meal         object    141836       0.01       4
                 carbs_g        float64    141836       0.01    1334
          chol

In [34]:
raw_df = read_diet_csv(RAW_PATH)
try:
    canon
except NameError:
    def canon(s: str) -> str:
        s = str(s).strip().lower()
        s = re.sub(r"[\s\-]+", "_", s)
        s = re.sub(r"[^a-z0-9_]", "", s)
        return s
cmap = {canon(c): c for c in raw_df.columns}

def find_col(cands):
    for c in cands:
        cc = canon(c)
        if cc in cmap:
            return cmap[cc]
    return None
raw_date_col = find_col(["date", "logged_date", "log_date", "added_on", "added on", "timestamp", "created_at"]) or "Date"
raw_dt = pd.to_datetime(raw_df[raw_date_col], errors="coerce")
bad_mask = raw_dt.isna() | (raw_dt.dt.year == 1969)
df_raw_filtered = raw_df[~bad_mask].copy()
print("Raw shape:", raw_df.shape, "Filtered shape:", df_raw_filtered.shape, "Removed:", int(bad_mask.sum()))
col_stats = []
for c in df_raw_filtered.columns:
    s = df_raw_filtered[c]
    col_stats.append({
        "column": c,
        "dtype": str(s.dtype),
        "non_null": int(s.notna().sum()),
        "missing_%": round(float(s.isna().mean()*100), 2),
        "unique": int(s.nunique(dropna=True)),
    })
quality = pd.DataFrame(col_stats).sort_values(["missing_%", "column"], ascending=[False, True])
print("Columns:", len(df_raw_filtered.columns))
print(quality.to_string(index=False))

Default read_csv failed, trying python engine: Error tokenizing data. C error: Expected 35 fields in line 8, saw 50

Raw shape: (148242, 35) Filtered shape: (148183, 35) Removed: 59
Columns: 35
                  column   dtype  non_null  missing_%  unique
              Brand Name  object     37402      74.76    3383
            Nix Brand Id  object     40378      72.75    3410
          Nix Brand Name  object     40378      72.75    3411
             Nix Item Id  object     40378      72.75   10697
           Nix Item Name  object     40377      72.75   10260
            Alt Measures  object    110232      25.61    5550
                 Og Unit  object    126443      14.67    1280
 Og Serving Weight Grams float64    146977       0.81    2421
Sel Serving Weight Grams float64    146977       0.81    2702
                     Qty  object    148152       0.02     936
                    Meal  object    148174       0.01       4
                Added On  object    148183       0.00   61000


In [35]:
nix_cols = [c for c in df_raw_filtered.columns if c.strip().lower().startswith("nix ")]
drop_cols = nix_cols + [c for c in df_raw_filtered.columns if c.strip().lower() == "brand name"]
df_raw_clean = df_raw_filtered.drop(columns=drop_cols, errors="ignore").copy()
rename_map = {c: re.sub(r"(?i)^nf\s+", "", c).strip().lower() for c in df_raw_clean.columns if c.strip().lower().startswith("nf ")}
df_raw_clean = df_raw_clean.rename(columns=rename_map)
print("Dropped:", drop_cols)
print("Renamed NF cols:", sorted(rename_map.values()))
print("Shape:", df_raw_clean.shape)
df_raw_clean.head(3)

Dropped: ['Nix Brand Id', 'Nix Brand Name', 'Nix Item Id', 'Nix Item Name', 'Brand Name']
Renamed NF cols: ['added sugars', 'calories', 'cholesterol', 'dietary fiber', 'potassium', 'protein', 'saturated fat', 'sodium', 'sugars', 'total carbohydrate', 'total fat', 'trans fatty acid']
Shape: (148183, 30)


,Diet Log ID,Participant name,Participant ID,Group,Date,Meal,Food Name,Thumb,Qty,Serving Type,calories,total fat,saturated fat,cholesterol,sodium,total carbohydrate,dietary fiber,sugars,protein,potassium,added sugars,trans fatty acid,Added On,Og Serving Weight Grams,Og Kcal,Og Unit,Sel Serving Qty,Sel Serving Weight Grams,Alt Measures,Full Nutrients
0,152340,user user 10,10,mLife+Points,06/30/2023,Lunch,deli turkey,https://nix-tag-images.s3.amazonaws.com/855_th...,3,oz,95.3,2.6,0.1,46.8,1020.6,6.5,0.3,3.4,11.5,296.8,11.5,11.5,06/30/2023 10:20:46 pm,56.7,63.50,oz,2.00,56.7,"[{""serving_weight"":48,""measure"":""oz (1 serving...","[{""attr_id"":203,""value"":7.6545},{""attr_id"":204..."
1,152341,user user 10,10,mLife+Points,06/30/2023,Lunch,fruit salad,https://nix-tag-images.s3.amazonaws.com/1773_t...,2,cup,193.2,0.9,0.1,0,5.3,49,6.5,32.9,2.7,728.3,2.7,0,06/30/2023 10:20:46 pm,162.4,96.62,cup,1.00,162.4,"[{""serving_weight"":162.4,""measure"":""cup"",""seq""...","[{""attr_id"":203,""value"":1.3555},{""attr_id"":204..."
2,152342,user user 10,10,mLife+Points,06/30/2023,Dinner,hummus,https://nix-tag-images.s3.amazonaws.com/636_th...,0.25,cup,99.6,5.8,0.9,0,227.4,8.6,3.6,4.7,4.7,136.8,4.7,4.7,06/30/2023 10:26:14 pm,60.0,99.60,cup,0.25,60.0,"[{""serving_weight"":15,""measure"":""tbsp"",""seq"":1...","[{""attr_id"":203,""value"":4.74},{""attr_id"":204,""..."


In [ ]:
meal_map = {"breakfast": 1, "lunch": 2, "dinner": 3, "snack": 4}
try:
    canon
except NameError:
    def canon(s: str) -> str:
        s = str(s).strip().lower()
        s = re.sub(r"[\s\-]+", "_", s)
        s = re.sub(r"[^a-z0-9_]", "", s)
        return s
if "df_raw_clean" in globals():
    colmap = {canon(c): c for c in df_raw_clean.columns}
    def find_col(cands):
        for c in cands:
            cc = canon(c)
            if cc in colmap:
                return colmap[cc]
        return None
    meal_col = find_col(["meal", "meal_type", "meal type", "meal_name", "category"])
    if meal_col is not None:
        df_raw_clean["meal_code"] = (
            df_raw_clean[meal_col].astype(str).str.strip().str.lower().map(meal_map).astype("Int64")
        )
        print(df_raw_clean["meal_code"].value_counts(dropna=False).sort_index())
    else:
        print("No meal-like column found in df_raw_clean.")

In [ ]:
try:
    canon
except NameError:
    import re
    def canon(s: str) -> str:
        s = str(s).strip().lower()
        s = re.sub(r"[\s\-]+", "_", s)
        s = re.sub(r"[^a-z0-9_]", "", s)
        return s

def _norm_meal(x):
    t = str(x).strip().lower()
    m = {"b": "breakfast", "breakfast": "breakfast", "l": "lunch", "lnch": "lunch", "lunch": "lunch", "d": "dinner", "din": "dinner", "dinner": "dinner", "snack": "snack", "snacks": "snack"}
    return m.get(t)

meal_map = {"breakfast": 1, "lunch": 2, "dinner": 3, "snack": 4}

if "df" in globals() and "meal" in df.columns:
    df["meal_code"] = df["meal"].map(lambda x: meal_map.get(_norm_meal(x))).astype("Int64")
    if "meal_code" in df.columns:
        print("df meal_code value counts:")
        print(df["meal_code"].value_counts(dropna=False).sort_index().to_string())

if "df_raw_clean" in globals():
    cmap = {canon(c): c for c in df_raw_clean.columns}
    meal_col = None
    for cand in ["meal", "meal_type", "meal name", "meal_name", "category"]:
        cc = canon(cand)
        if cc in cmap:
            meal_col = cmap[cc]
            break
    if meal_col is not None:
        df_raw_clean["meal_code"] = df_raw_clean[meal_col].map(lambda x: meal_map.get(_norm_meal(x))).astype("Int64")
        print("df_raw_clean meal_code value counts:")
        print(df_raw_clean["meal_code"].value_counts(dropna=False).sort_index().to_string())

In [37]:
def save_filtered_outputs():
    os.makedirs(CLEAN_DIR, exist_ok=True)
    # Row-level export (exclude unparsed dates and 1969)
    if 'df' in globals():
        cols_base = [c for c in ["participant_id", "datetime_raw", "date", "year", "month", "dow", "meal", "meal_code", "food_name", "qty", "serving_type"] if c in df.columns]
        cols_nutr = [c for c in ["kcal", "protein_g", "carbs_g", "fat_g", "sugars_g", "fiber_g", "sodium_mg", "cholesterol_mg", "potassium_mg"] if c in df.columns]
        keep_cols = cols_base + cols_nutr
        df_out = df.loc[df.get("date").notna() & (df.get("year") != 1969)].copy() if ("date" in df.columns and "year" in df.columns) else df.copy()
        rowlevel_out = os.path.join(CLEAN_DIR, "diet_log_rowlevel_cleaned_v2.csv")
        df_out[keep_cols].to_csv(rowlevel_out, index=False)
        print("row-level:", rowlevel_out, len(df_out), "rows", len(keep_cols), "cols")
    else:
        print("df not defined; skipping row-level export")

    # Daily aggregates export (exclude unparsed and 1969)
    if 'daily' in globals():
        if all(c in daily.columns for c in ["date", "year"]):
            daily_out_df = daily.loc[daily["date"].notna() & (daily["year"] != 1969)].copy()
        else:
            daily_out_df = daily.copy()
        daily_out = os.path.join(CLEAN_DIR, "diet_log_daily_ml_ready.csv")
        daily_out_df.to_csv(daily_out, index=False)
        print("daily:", daily_out, len(daily_out_df), "rows")
    else:
        print("daily not defined; skipping daily export")

    # Save raw pruned/renamed dataset
    if 'df_raw_clean' in globals():
        raw_pruned_out = os.path.join(CLEAN_DIR, "diet_log_raw_pruned.csv")
        df_raw_clean.to_csv(raw_pruned_out, index=False)
        print("raw-pruned:", raw_pruned_out, len(df_raw_clean), "rows", df_raw_clean.shape[1], "cols")
    else:
        print("df_raw_clean not defined; skipping raw-pruned export")

    # Export column-quality report for df_raw_filtered
    if 'df_raw_filtered' in globals():
        reports_dir = os.path.join(CLEAN_DIR, "reports")
        os.makedirs(reports_dir, exist_ok=True)
        total = len(df_raw_filtered)
        rows = []
        for c in df_raw_filtered.columns:
            s = df_raw_filtered[c]
            non_null = int(s.notna().sum())
            missing = total - non_null
            nunique = int(s.nunique(dropna=True))
            rows.append({
                "column": c,
                "dtype": str(s.dtype),
                "non_null": non_null,
                "total": total,
                "missing": missing,
                "missing_pct": round((missing / total) * 100, 3) if total else 0.0,
                "nunique": nunique,
            })
        qual_df = pd.DataFrame(rows).sort_values(["missing_pct", "nunique", "column"], ascending=[True, False, True])
        qual_out = os.path.join(reports_dir, "diet_log_raw_filtered_column_quality.csv")
        qual_df.to_csv(qual_out, index=False)
        print("raw-filtered quality:", qual_out, len(qual_df), "rows")
    else:
        print("df_raw_filtered not defined; skipping column-quality export")

print("Ready: call save_filtered_outputs() to write cleaned artifacts.")

Ready: call save_filtered_outputs() to write cleaned artifacts.


In [ ]:
print("No files will be written automatically. To export, run: enable_writes(True); save_filtered_outputs(out_dir='tmp')")ts(out_dir="tmp"):
    base_dir = TMP_DIR if str(out_dir).lower() == "tmp" else CLEAN_DIR
    os.makedirs(base_dir, exist_ok=True)

    if not WRITE_ENABLED:
        print("[SKIP WRITE] WRITE_ENABLED is False; no files will be created.")
        return

    # Row-level export (exclude unparsed dates and 1969)
    if 'df' in globals():
        cols_base = [c for c in ["participant_id", "datetime_raw", "date", "year", "month", "dow", "meal", "meal_code", "food_name", "qty", "serving_type"] if c in df.columns]
        cols_nutr = [c for c in ["kcal", "protein_g", "carbs_g", "fat_g", "sugars_g", "fiber_g", "sodium_mg", "cholesterol_mg", "potassium_mg"] if c in df.columns]
        keep_cols = cols_base + cols_nutr
        df_out = df.loc[df.get("date").notna() & (df.get("year") != 1969)].copy() if ("date" in df.columns and "year" in df.columns) else df.copy()
        rowlevel_out = os.path.join(base_dir, "diet_log_rowlevel_cleaned_v2.csv")
        df_out[keep_cols].to_csv(rowlevel_out, index=False)
        print("row-level ->", rowlevel_out, len(df_out), "rows", len(keep_cols), "cols")
    else:
        print("df not defined; skipping row-level export")

    # Daily aggregates export (exclude unparsed and 1969)
    if 'daily' in globals():
        if all(c in daily.columns for c in ["date", "year"]):
            daily_out_df = daily.loc[daily["date"].notna() & (daily["year"] != 1969)].copy()
        else:
            daily_out_df = daily.copy()
        daily_out = os.path.join(base_dir, "diet_log_daily_ml_ready.csv")
        daily_out_df.to_csv(daily_out, index=False)
        print("daily ->", daily_out, len(daily_out_df), "rows")
    else:
        print("daily not defined; skipping daily export")

    # Save raw pruned/renamed dataset
    if 'df_raw_clean' in globals():
        raw_pruned_out = os.path.join(base_dir, "diet_log_raw_pruned.csv")
        df_raw_clean.to_csv(raw_pruned_out, index=False)
        print("raw-pruned ->", raw_pruned_out, len(df_raw_clean), "rows", df_raw_clean.shape[1], "cols")
    else:
        print("df_raw_clean not defined; skipping raw-pruned export")

    # Export column-quality report for df_raw_filtered
    if 'df_raw_filtered' in globals():
        reports_dir = os.path.join(base_dir, "reports")
        os.makedirs(reports_dir, exist_ok=True)
        total = len(df_raw_filtered)
        rows = []
        for c in df_raw_filtered.columns:
            s = df_raw_filtered[c]
            non_null = int(s.notna().sum())
            missing = total - non_null
            nunique = int(s.nunique(dropna=True))
            rows.append({
                "column": c,
                "dtype": str(s.dtype),
                "non_null": non_null,
                "total": total,
                "missing": missing,
                "missing_pct": round((missing / total) * 100, 3) if total else 0.0,
                "nunique": nunique,
            })
        qual_df = pd.DataFrame(rows).sort_values(["missing_pct", "nunique", "column"], ascending=[True, False, True])
        qual_out = os.path.join(reports_dir, "diet_log_raw_filtered_column_quality.csv")
        qual_df.to_csv(qual_out, index=False)
        print("raw-filtered quality ->", qual_out, len(qual_df), "rows")
    else:
        print("df_raw_filtered not defined; skipping column-quality export")

print("Ready: writes are disabled by default. Call enable_writes(True) and save_filtered_outputs(out_dir='tmp') to export to tmp.")

row-level: /Users/clee/Documents/Lab/mlife/data/cleaned/diet_log_rowlevel_cleaned_v2.csv 141791 rows 19 cols
daily: /Users/clee/Documents/Lab/mlife/data/cleaned/diet_log_daily_ml_ready.csv 18824 rows
raw-pruned: /Users/clee/Documents/Lab/mlife/data/cleaned/diet_log_raw_pruned.csv 148183 rows 30 cols
raw-filtered quality: /Users/clee/Documents/Lab/mlife/data/cleaned/reports/diet_log_raw_filtered_column_quality.csv 35 rows
